In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

!pip install transformers
!pip install bert-tensorflow

In [20]:
from google.colab import drive
drive.mount('/content/gdrive') 
model_save_name = 'haspeede_test_ufficiale.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 

Mounted at /content/gdrive


#Verbalization and davinci

In [21]:
train = pd.read_excel('haspeedee_train.xlsx') 
dev = pd.read_excel('haspeedee_test.xlsx')
conan = pd.read_excel('CONAN Italian.xlsx')
data = pd.concat([train,dev])
data = data.rename(columns={'anonymized_tweet_id':'s'})
data = data.drop(columns=['data_type', 'dataset'])
kg_verbalized = pd.read_excel('kg4haspeede_verbalised.xlsx')

In [22]:
conan = conan.rename(columns={'cn_id':'s', 'hateSpeech':'text', 'hsType':'label'})
conan['label'] = 1
conan

,s,text,label
0,ITT1ST0001HS0016CN000010,"Li accogliamo a casa nostra, li manteniamo e l...",1
1,ITT1ST0001HS0016CN000010P1,Noi accogliamo i musulmani e provvediamo a lor...,1
2,ITT1ST0001HS0016CN000010P2,"Noi li ospitiamo nel nostro paese, forniamo un...",1
3,ITT1ST0001HS0016CN000011,"Li accogliamo a casa nostra, li manteniamo e l...",1
4,ITT1ST0001HS0016CN000011P1,Noi accogliamo i musulmani e provvediamo a lor...,1
...,...,...,...
3160,ITT1ST0023HS0055CN001068P1,Una donna che permette di farsi coprire con un...,1
3161,ITT1ST0023HS0055CN001068P2,In Italia non possiamo accogliere donne che de...,1
3162,ITT1ST0023HS0055CN001069,Una donna che accetta di vivere nascosta sotto...,1
3163,ITT1ST0023HS0055CN001069P1,Una donna che permette di farsi coprire con un...,1


In [23]:
#just verbalization
#let's merge the two dataset on the column 'id'
merged_df = pd.merge(data, kg_verbalized[['s', 'triples_verbalised']], on='s') 
#merged_df
#join text and triples
merged_df['text'] = merged_df['anonymized_text'].str.cat(merged_df['triples_verbalised'], sep=' ')
merged_df
#concatenate with the other 'data'
data = data.rename(columns={'anonymized_text':'text'})
merged_df = merged_df.drop(columns=['anonymized_text', 'triples_verbalised'])
# merge the two datasets based on the 'id' column
merged = pd.concat([merged_df, data])
unique = merged.drop_duplicates(subset='s', keep='first')
data = unique

In [24]:
#add conan
data = pd.concat([data,conan])
data

,s,label,text
0,217874450618134,1,@unknown Con tutte le denunce che si sta becca...
1,740684329660693,0,In pochi giorni il #ministro #salvini ha butta...
2,170378636015491,1,Ma se la #carfagna e #crosetto sono più a #sin...
3,906842219275344,1,Nemmeno la tradizionale conferenza stampa di f...
5,48581434415291,1,#decretosicurezza #LeolucaOrlando colui che er...
...,...,...,...
3160,ITT1ST0023HS0055CN001068P1,1,Una donna che permette di farsi coprire con un...
3161,ITT1ST0023HS0055CN001068P2,1,In Italia non possiamo accogliere donne che de...
3162,ITT1ST0023HS0055CN001069,1,Una donna che accetta di vivere nascosta sotto...
3163,ITT1ST0023HS0055CN001069P1,1,Una donna che permette di farsi coprire con un...


In [ ]:
#to add davinci
davinci = pd.read_excel('haspeede_verbalis_davinci.xlsx')
#davinci = davinci.drop(columns=['o', 'odang_person', 'triples_verbalised', 'name', 'from_LM'])
#concateno text e triples_verbalised
#davinci['text'] = davinci['text'].str.cat(davinci['triples_verbalised'], sep=' ')
#davinci = davinci.drop(columns=['Unnamed: 0', 'o', 'odang_person', 'triples_verbalised', 'name', 'from_LM'])
train = pd.read_excel('haspeedee_train.xlsx')
train.rename(columns={'anonymized_tweet_id':'s'}, inplace=True)
train.rename(columns={'anonymized_text':'text'}, inplace=True)
train.drop(columns=['dataset', 'data_type'], inplace=True)

def replace_text(davinci, train):
    # Merge the 'train' and 'davinci' datasets based on 'ID'
    merged_dataset = pd.merge(train, davinci[['s', 'text']], on='s', how='left')

    # Update the 'text' column of 'train' with the values from 'davinci' where IDs match
    merged_dataset['text'] = merged_dataset['text_y'].combine_first(merged_dataset['text_x'])

    # Drop the extra columns
    merged_dataset = merged_dataset.drop(['text_x', 'text_y'], axis=1)

    return merged_dataset

    
merged_data = replace_text(davinci, train)
merged_data.drop_duplicates(subset='s', keep='first', inplace=True)
data = merged_data

#Training misogyny

In [ ]:
#if you want to train the baseline without adding material
data = pd.read_excel('haspeedee_train.xlsx')
data.rename(columns={'anonymized_tweet_id':'s'}, inplace=True)
data.rename(columns={'anonymized_text':'text'}, inplace=True)
data.drop(columns=['dataset', 'data_type'], inplace=True)

In [ ]:
#if you want to add the definition of HS
data['text'] = data['text'].apply(lambda x: x + 'Per discorso d\'odio si intende qualsiasi tipo di comunicazione che attacchi o utilizzi un linguaggio peggiorativo o discriminatorio nei confronti di una persona o di un gruppo in base alla sua identità, ovvero in base alla sua religione, etnia, nazionalità, razza, colore, discendenza, sesso o altro fattore di identità.')

In [25]:
from sklearn.model_selection import train_test_split

# Split dataset in traning and validation(test)
X_train, X_val, Y_train, Y_val = train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.10,
    random_state=17,
    stratify=data.label.values
)

In [ ]:
# Check datasets composition
data['data_type'] = ['not_set'] * data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.groupby(['label', 'data_type']).count()

In [ ]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import TensorDataset


tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0", use_fast=False)

In [ ]:
# Encode training dataset using the tokenizer
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True, 
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

# Encode validation dataset using the tokenizer
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type == 'val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

In [29]:
# Extract IDs, attention masks and labels from training dataset
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type == 'train'].label.values)
labels_train

tensor([1, 1, 1,  ..., 1, 1, 1])

In [30]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type == 'val'].label.values)

In [31]:
# Create train and validation dataset from extracted features
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
print("Train dataset length: {}\nValidation dataset length: {}".format(len(dataset_train), len(dataset_val)))

Train dataset length: 6855
Validation dataset length: 1910


In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Define the size of each batch
batch_size = 16

# Load training dataset
dataloader_train= DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size)

# Load valuation dataset
dataloader_val= DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Define model optimizer -> Adam
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, 
    eps=1e-8
)
# Define model scheduler
epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [35]:
import random

# Define random seeds
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
# Define processor type for torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [37]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Returns the F1 score computed on the predictions
def f1_score_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')



# Returns the precision and recall score computed on the predictions
def prec_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return precision_score(labels_flat, preds_flat, average='macro')

def recall_func(preds, labels):
    preds_flat=np.argmax(preds, axis=1).flatten()
    labels_flat=labels.flatten()
    return recall_score(labels_flat, preds_flat, average='macro')



In [38]:
# Evaluates the model using the validation set
def evaluate(dataloader_val):
  model.eval()
  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in dataloader_val:
      batch = tuple(b.to(device) for b in batch)
      inputs = {'input_ids': batch[0],
        'attention_mask': batch[1],
        'labels': batch[2],
        }

      with torch.no_grad():
          outputs = model(**inputs)

      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)

  loss_val_avg = loss_val_total / len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs + 1)):

    model.train()  # model is training

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        
        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()  # to backpropagate

        torch.nn.utils.clip_grad_norm_(model.parameters(),
                                      1.0)  # prevents the gradient from being too small or too big

        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    
    torch.save(model, path)
    tqdm.write(f'\nEpoch {epoch}/{epochs}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')  # make sure that model is still training

    val_loss, predictions, true_vals = evaluate(dataloader_val)  # to check overtraining (or overfitting)
    val_f1 = f1_score_func(predictions, true_vals)
    val_prec = prec_func(predictions, true_vals)
    val_recall = recall_func(predictions, true_vals)

    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score(weighted) : {val_f1}')
    tqdm.write(f'Prec Score(weighted) : {val_prec}')
    tqdm.write(f'Recall Score(weighted) : {val_recall}')


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/429 [00:00<?, ?it/s]


Epoch 1/4
Training loss: 0.25287135294475754
Validation loss: 0.198703194106929
F1 Score(weighted) : 0.9289142429239547
Prec Score(weighted) : 0.9274437377390707
Recall Score(weighted) : 0.9307222551875427


Epoch 2:   0%|          | 0/429 [00:00<?, ?it/s]

In [ ]:
model.save_pretrained('alberto_haspeede_verbalizz_conan_testsetuff')

#Preds sul test

In [ ]:
#Load test set
data_test = pd.read_csv('haspeede_test_subtaskB.csv')
data_test

In [ ]:
data_test = data_test.rename(columns={'anonymized_tweet_id':'s', 'anonymized_text':'text'})
data_test = data_test.drop(columns=['dataset'])

In [ ]:
merged_datatest = replace_text(davinci, data_test)
merged_datatest.drop_duplicates(subset='s', keep='first', inplace=True)
data_test = merged_datatest

In [ ]:
#definizione
data_test['text'] = data_test['text'].apply(lambda x: x + 'Per discorso d\'odio si intende qualsiasi tipo di comunicazione che attacchi o utilizzi un linguaggio peggiorativo o discriminatorio nei confronti di una persona o di un gruppo in base alla sua identità, ovvero in base alla sua religione, etnia, nazionalità, razza, colore, discendenza, sesso o altro fattore di identità.')

In [ ]:
# Encode validation dataset using the tokenizer
encoded_data_test = tokenizer.batch_encode_plus(
    data_test.text.values,
    add_special_tokens=True,
    return_attention_mask=True,  
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


In [ ]:
# Extract IDs, attention masks and labels from validation dataset
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)
print("Test dataset length: {}".format(len(dataset_test)))

In [ ]:
from torch.utils.data import DataLoader
dataloader_test = DataLoader(dataset_test)

In [ ]:
# Evaluates the model using the validation set
def predict(dataset_test):
    predictions = []

    for row in dataset_test:
      row = tuple(r.to(device) for r in row)
      inputs = {'input_ids': row[0],
        'attention_mask': row[1]
        }

      with torch.no_grad():
          outputs = model(**inputs)

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictions.append(logits)

    return predictions

# Predict values for test dataset
predictions = predict(dataloader_test)

In [ ]:
print(len(predictions))
results = []
for i, prediction in enumerate(predictions):
  predicted = np.argmax(prediction, axis=1)[0]
  # print(f"index: {i} -- prediction: {predicted}")
  results.append(predicted)

print(results)
print(results.count(0))
print(results.count(1))

In [ ]:
misogynous = []
for prediction in results:
  misogynous.append(prediction)

In [ ]:
#Predict misogyny
data_test['preds'] = misogynous


In [ ]:
#Save the predicted labels in a csv file 
data_test.to_excel('preds_haspeedee_taskB_verb+conan.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('preds_haspeedee_taskB_verb+conan.xlsx')

#Testing 

In [ ]:
import pandas as pd
import sklearn
from sklearn.metrics import classification_report

In [ ]:
y_true = data_test['label'].to_numpy()
y_preds = data_test['preds'].to_numpy()
print(sklearn.metrics.classification_report(y_true, y_preds))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89       346
           1       0.79      0.93      0.85       214

    accuracy                           0.88       560
   macro avg       0.87      0.89      0.87       560
weighted avg       0.89      0.88      0.88       560

